In [0]:
from pyspark.sql.functions import current_timestamp, lit
from datetime import datetime

print(" Notebook Bronze - Ingestão da fato iniciada")

In [0]:
processed_date = datetime.now().strftime("%Y-%m-%d")
failed_files = []
processed_files = []

try:
    files = dbutils.fs.ls("abfss://landing@mystoacc.dfs.core.windows.net/")
    fact_files = [f.name.replace('.csv', '') for f in files 
                 if f.name.startswith('fato_')]
    
    print(f"Arquivos de fato encontrados na landing: {fact_files}")
    
    if len(fact_files) == 0:
        print("Nenhum arquivo de fato encontrado. Finalizando.")
    
except Exception as e:
    print(f"Erro ao listar landing: {e}")
    fact_files = []

for table_name in fact_files:
    try:
        print(f"\nProcessando {table_name}...")

        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv")
        
        df_ct = df.withColumn("ingestion_timestamp", current_timestamp()) \
                  .withColumn("ingestion_date", lit(processed_date))

        df_ct.write.format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .save(f"abfss://bronze@mystoacc.dfs.core.windows.net/facts/{table_name}")

        print(f"tabela fato: {table_name} salva na bronze/facts/")

        source = f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv"
        dest = f"abfss://landing@mystoacc.dfs.core.windows.net/processed/facts/{processed_date}/{table_name}.csv"
        dbutils.fs.cp(source, dest)
        dbutils.fs.rm(source)

        print(f"arquivo: {table_name} movido para processed/facts/")
        processed_files.append(table_name)

    except Exception as e:
        print(f"ERRO: {str(e)}")
        failed_files.append(table_name)

        source = f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv"
        dest = f"abfss://landing@mystoacc.dfs.core.windows.net/failed/facts/{processed_date}/{table_name}.csv"
        try:
            dbutils.fs.cp(source, dest)
            dbutils.fs.rm(source)
            print(f"Arquivo: {table_name} movido para failed/facts/")
        except:
            print(f"Nao foi possivel mover arquivo")

print(f"\narquivos processados: {processed_files}")
print(f"arquivos com falha: {failed_files}")